In [ ]:
import requests
from bs4 import BeautifulSoup
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, SystemMessage
from typing import Dict, List, cast, Optional, Any
from langchain_core.tools import tool
from langchain.agents import AgentExecutor
from langchain_core.prompts import PromptTemplate

from dataclasses import dataclass, field
from typing import Annotated, Sequence
from langgraph.graph import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage

@dataclass
class State:
    session_id: str = ""
    github_url: str = ""
    company_name: str = ""
    messages: Annotated[Sequence[AnyMessage], add_messages] = field(default_factory=list)

# 세션 초기화 (스크래핑용)
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
})

def _scrape_page(url: str) -> Optional[BeautifulSoup]:
    """웹 페이지 스크래핑 헬퍼 함수"""
    try:
        r = session.get(url, timeout=10)
        if r.status_code == 200:
            return BeautifulSoup(r.text, 'html.parser')
        return None
    except:
        return None

@tool
def scrape_user_profile(username: str) -> Dict[str, Any]:
    """
    Scrape GitHub user profile information from their main page
    
    Args:
        username: GitHub username
    
    Returns:
        사용자 프로필 정보 딕셔너리
    """
    soup = _scrape_page(f"https://github.com/{username}")
    if not soup:
        return {"error": "Failed to scrape profile"}

    result = {
        "username": username, "name": None, "bio": None, "location": None,
        "company": None, "followers": None, "following": None, "repositories": None
    }

    name_elem = soup.select_one('.vcard-fullname')
    if name_elem: result["name"] = name_elem.text.strip()

    bio_elem = soup.select_one('.user-profile-bio > div')
    if bio_elem: result["bio"] = bio_elem.text.strip()

    for item in soup.select('.vcard-detail'):
        text = item.text.strip()
        if 'location' in str(item):
            result["location"] = text
        elif 'organization' in str(item):
            result["company"] = text

    for link in soup.select('a.Link--secondary'):
        href = link.get('href', '')
        text = link.text.strip()
        if 'followers' in href:
            result["followers"] = text.split()[0]
        elif 'following' in href:
            result["following"] = text.split()[0]

    repo_count = soup.select_one('span.Counter')
    if repo_count:
        result["repositories"] = repo_count.text.strip()

    return result

@tool
def scrape_user_repos(username: str, page: int = 1) -> Dict[str, Any]:
    """
    Scrape list of repositories from user's repositories tab
    
    Args:
        username: GitHub username
        page: Page number for pagination
    
    Returns:
        Dictionary containing list of repositories and count
    """
    url = f"https://github.com/{username}?tab=repositories"
    if page > 1: url += f"&page={page}"
    soup = _scrape_page(url)
    if not soup:
        return {"error": "Failed to scrape repositories"}

    repos = []
    for repo_elem in soup.select('#user-repositories-list > ul > li'):
        repo_data = {}
        name_elem = repo_elem.select_one('a[itemprop="name codeRepository"]')
        if name_elem:
            repo_data["name"] = name_elem.text.strip()
            repo_data["url"] = f"https://github.com{name_elem.get('href','')}"
        desc_elem = repo_elem.select_one('p[itemprop="description"]')
        if desc_elem: repo_data["description"] = desc_elem.text.strip()
        lang_elem = repo_elem.select_one('[itemprop="programmingLanguage"]')
        if lang_elem: repo_data["language"] = lang_elem.text.strip()
        star_elem = repo_elem.select_one('a[href*="stargazers"]')
        if star_elem: repo_data["stars"] = star_elem.text.strip()
        fork_elem = repo_elem.select_one('a[href*="forks"]')
        if fork_elem: repo_data["forks"] = fork_elem.text.strip()
        time_elem = repo_elem.select_one('relative-time')
        if time_elem: repo_data["updated"] = time_elem.get('datetime', '')
        if repo_data.get("name"): repos.append(repo_data)
    return {"repositories": repos, "count": len(repos)}

@tool
def scrape_repo_main_page(owner: str, repo: str) -> Dict[str, Any]:
    """
    Scrape repository main page for basic info and README
    
    Args:
        owner: Repository owner username
        repo: Repository name
    
    Returns:
        Dictionary containing repository info, README, etc.
    """
    soup = _scrape_page(f"https://github.com/{owner}/{repo}")
    if not soup:
        return {"error": "Failed to scrape repository"}

    result = {
        "name": repo, "owner": owner, "description": None, "stars": None,
        "forks": None, "watching": None, "readme": None, "languages": {}, "about": {}
    }

    desc_elem = soup.select_one('p.f4')
    if desc_elem: result["description"] = desc_elem.text.strip()

    for elem in soup.select('#repo-content-pjax-container a.Link--muted'):
        href = elem.get('href',''); text = elem.text.strip()
        if '/stargazers' in href: result["stars"] = text.split()[0]
        elif '/forks' in href: result["forks"] = text.split()[0]
        elif '/watchers' in href: result["watching"] = text.split()[0]

    readme_elem = soup.select_one('article.markdown-body')
    if readme_elem:
        readme_text = readme_elem.get_text(separator='\n', strip=True)
        result["readme"] = readme_text[:3000]

    for lang_elem in soup.select('.BorderGrid-cell span[itemprop="programmingLanguage"]'):
        lang = lang_elem.text.strip()
        percent_elem = lang_elem.find_next('span', class_='ml-1')
        if percent_elem:
            result["languages"][lang] = percent_elem.text.strip()

    about_elem = soup.select_one('.BorderGrid')
    if about_elem:
        for link in about_elem.select('a'):
            text = link.text.strip()
            if 'release' in link.get('href',''):
                result["about"]["releases"] = text
            elif 'commit' in link.get('href',''):
                result["about"]["commits"] = text
            elif 'contributor' in link.get('href',''):
                result["about"]["contributors"] = text

    return result

@tool
def scrape_repo_commits(owner: str, repo: str) -> Dict[str, Any]:
    """
    Scrape recent commits from repository
    
    Args:
        owner: Repository owner username
        repo: Repository name
    
    Returns:
        Dictionary containing list of commits and count
    """
    soup = _scrape_page(f"https://github.com/{owner}/{repo}/commits")
    if not soup:
        return {"error": "Failed to scrape commits"}

    commits = []
    for commit_elem in soup.select('.Box-row')[:10]:
        commit_data = {}
        msg_elem = commit_elem.select_one('a.Link--primary')
        if msg_elem: commit_data["message"] = msg_elem.text.strip()
        author_elem = commit_elem.select_one('a.commit-author')
        if author_elem: commit_data["author"] = author_elem.text.strip()
        time_elem = commit_elem.select_one('relative-time')
        if time_elem: commit_data["date"] = time_elem.get('datetime','')
        hash_elem = commit_elem.select_one('a.text-mono')
        if hash_elem: commit_data["sha"] = hash_elem.text.strip()
        if commit_data: commits.append(commit_data)
    return {"commits": commits, "count": len(commits)}

@tool
def scrape_repo_files(owner: str, repo: str) -> Dict[str, Any]:
    """
    Scrape repository file structure
    
    Args:
        owner: Repository owner username
        repo: Repository name
    
    Returns:
        Dictionary containing list of files and count
    """
    soup = _scrape_page(f"https://github.com/{owner}/{repo}")
    if not soup:
        return {"error": "Failed to scrape files"}
    files = []
    for row in soup.select('div[role="row"]'):
        name_elem = row.select_one('a.Link--primary')
        if name_elem:
            name = name_elem.text.strip()
            icon = row.select_one('svg')
            file_type = "file"
            if icon and 'directory' in str(icon):
                file_type = "directory"
            files.append({"name": name, "type": file_type})
    return {"files": files, "count": len(files)}


In [3]:
scrape_repo_files.description

'Scrape repository file structure\n\nArgs:\n    owner: Repository owner username\n    repo: Repository name\n\nReturns:\n    Dictionary containing list of files and count'

In [4]:

async def github_agent(state: State) -> Dict[str, List[AIMessage]]:
    """
    Analyze GitHub profile and repository information
    """

    AGENT_MODEL="gpt-4o-mini"

    model = ChatOpenAI(model=AGENT_MODEL, temperature=0, api_key=OPENAI_API_KEY)

    username = state.github_url.rstrip('/').split('/')[-1] if 'github.com/' in state.github_url else state.github_url
    print("username: ", username)
    # LangChain tool 데코레이터로 정의된 도구들을 사용
    tools = [
        scrape_user_profile,
        scrape_user_repos,
        scrape_repo_main_page,
        scrape_repo_commits,
        scrape_repo_files
    ]

    system_message = f"""You're an expert at analyzing GitHub repositories.
    You use web scraping tools to analyze GitHub profiles and repositories.
    
    username: {username}

    You are a GitHub repository analyst with web scraping tools.
    Analyze GitHub profiles and repositories by scraping public web pages.
    Always respond in Korean.
    
    Available scraping tools:
    - scrape_user_profile: Get user profile information
    - scrape_user_repos: Get list of repositories
    - scrape_repo_main_page: Get repository details and README
    - scrape_repo_commits: Get recent commits
    - scrape_repo_files: Get file structure
    
    IMPORTANT INSTRUCTIONS:
    1. Use tools strategically to gather information (maximum 3-4 tool calls)
    2. After collecting sufficient information, provide a comprehensive Korean summary
    3. Do NOT continue using tools indefinitely - stop when you have enough information
    4. Your final response should be a complete analysis in Korean, not a request for more tools
    
    Process:
    1. Start with scrape_user_profile or scrape_repo_main_page for basic info
    2. Use 1-2 additional tools if needed for specific details
    3. Provide final comprehensive summary in Korean and STOP
    """

    # prompt = PromptTemplate.from_template(system_message)
    agent = create_react_agent(model, tools, prompt=system_message)

    # recursion_limit 설정으로 무한 루프 방지
    config = {"recursion_limit": 10, "max_iterations": 5}
    response = await agent.ainvoke({"messages": state.messages}, config=config)
    print("response: ", response)
    
    return {
        "messages": [response["messages"][-1]],
        "agent_name": "github_agent",
        "company_summary": response["messages"][-1].content,
    }


In [5]:
# 테스트 케이스들
test_cases = [
    {
        "description": "테스트 1: 사용자 프로필 분석",
        "github_url": "https://github.com/Pseudo-Lab",
        "query": "JobPT 레포에 대해서 요약해줘",
    },
    {
        "description": "테스트 2: 저장소 상세 분석",
        "github_url": "https://github.com/Pseudo-Lab/JobPT",
        "query": "이 저장소의 README, 최근 커밋, 사용 언어를 분석해주세요.",
    },
    {
        "description": "테스트 3: 조직 분석",
        "github_url": "https://github.com/Pseudo-Lab",
        "query": "이 조직의 주요 오픈소스 프로젝트들을 파악하고, 어떤 기술 스택을 사용하는지 분석해주세요.",
    },
]

state = State(
    session_id=f"test_session_1",
    github_url=test_cases[0]['github_url'],
    company_name=test_cases[0]['github_url'].split('/')[-1],  # URL에서 마지막 부분을 이름으로 사용
    messages=[HumanMessage(content=test_cases[0]['query'])],
)


In [6]:
result = await github_agent(state)

username:  Pseudo-Lab
response:  {'messages': [HumanMessage(content='JobPT 레포에 대해서 요약해줘', additional_kwargs={}, response_metadata={}, id='5be3501a-aaab-435b-a90c-b9e1c540d2a9'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xK8vkoHOXQVWGIQbE1fTFUkJ', 'function': {'arguments': '{"username":"Pseudo-Lab"}', 'name': 'scrape_user_repos'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 566, 'total_tokens': 585, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CXnG7ZpDgqItQhdwsLU3Z3pxxCtfK', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--83b9ca82-bdcf-4e39-a6d8-32b2caa34041-0', tool_calls=[{'name': 'scrape_user_repos', 'args': {'username': 'P

In [7]:
result['company_summary']

'**JobPT 레포지토리 요약**\n\n**레포지토리 이름:** JobPT  \n**소유자:** Pseudo-Lab  \n**스타 수:** 32  \n**포크 수:** 5  \n**설명:** 구직자의 이력서와 채용 공고를 매칭해주고, 채용 공고에 맞는 이력서를 검색하여 추천해주는 채용공고 매칭 시스템입니다.\n\n### 서비스 개요\n현대 사회에서 취업 준비는 반복적이고 시간 소모적인 과정으로, 구직자가 자신의 역량을 효과적으로 어필하지 못해 기회를 놓치는 경우가 많습니다. 이를 해결하기 위해 JobPT는 LLM(대형 언어 모델) 기반의 지능형 취업 지원 서비스를 제공합니다. 주요 기능은 다음과 같습니다:\n\n1. **채용 공고 추천:** 제공된 이력서를 RAG(회귀적 어텐션 그래프)를 활용하여 각 회사의 채용 공고와 의미론적 유사도를 기반으로 적합한 회사를 추천합니다.\n2. **이력서 피드백:** 유저와의 대화를 통해 이력서를 반복적으로 개선하는 프로세스를 제공합니다.\n3. **회사 정보 요약:** AI Agent를 통해 원하는 회사 또는 추천된 JD(직무 설명)의 최신 정보를 요약해 제공합니다.\n4. **이력서 적합성 평가:** 이력서를 분석하여 지원자 추적 시스템(ATS)이 이를 어떻게 평가할지 시뮬레이션합니다.\n\n### 기술 스택\n- **프로그래밍 언어:** Python, Typescript\n- **LLM 프레임워크:** LangChain, Langgraph, OpenAI\n- **백엔드:** FastAPI\n- **벡터 데이터베이스:** Pinecone\n- **모니터링:** Langfuse\n\n### 시스템 구성\n- **크롤링 API:** 여러 회사 JD 데이터 확보 및 마감된 채용 공고 삭제\n- **PDF 파싱:** 다양한 형태의 이력서 PDF 텍스트 추출\n- **AI Agent:** 멀티 에이전트로 파이프라인 구성\n\n### 실행 방법\n- Docker를 이용한 실행 및 로컬 환경에서의 직접 실행 방법이 제공됩니다. AP